In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
#from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
#from tensorflow.keras.models import Model

In [113]:
lines= pd.read_table('mar1.txt', names=['eng', 'fre'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [114]:
!pip install bpemb

In [0]:
from bpemb import BPEmb

In [0]:
lines.fre

0                               Va !
1                            Cours !
2                           Courez !
3                         Ça alors !
4                           Au feu !
5                         À l'aide !
6                             Saute.
7                        Ça suffit !
8                             Stop !
9                       Arrête-toi !
10                         Attends !
11                        Attendez !
12                     Je comprends.
13                         J'essaye.
14                      J'ai gagné !
15                 Je l'ai emporté !
16                          Oh non !
17                         Attaque !
18                        Attaquez !
19                           Santé !
20                   À votre santé !
21                           Merci !
22                         Lève-toi.
23                       J'ai pigé !
24                         Compris !
25                            Pigé ?
26                         Compris ?
2

In [0]:
type(lines.eng)

pandas.core.series.Series

In [116]:
bpemb_en = BPEmb(lang="en")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
bpemb_fr = BPEmb(lang="fr")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
## Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.fre=lines.fre.apply(lambda x: x.lower())

In [ ]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.fre=lines.fre.apply(lambda x: re.sub("'", '', x))

In [ ]:
import string

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.fre=lines.fre.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.fre = lines.fre.apply(lambda x: re.sub("[0123456789]", "", x))

In [ ]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.fre=lines.fre.apply(lambda x: x.strip())
print(lines.eng)
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.fre=lines.fre.apply(lambda x: re.sub(" +", " ", x))

In [ ]:
# Add $ and # tokens to target sequences
lines.fre = lines.fre.apply(lambda x : '$_ '+ x + ' _#')

# DOING THE BYTE PER ENCODING OF SOURCE SIDE SENTENCES

In [0]:
subword_eng=bpemb_en.encode(lines.eng)

In [ ]:
import pickle

In [0]:
#Making the Pickle File of subword
pickle.dump(subword_eng,open("sub_only_eng.pkl","wb"))

In [0]:
all_sub_eng_vocab=pickle.load(open("all_sub_eng_vocab.pkl","rb"))

In [0]:
sub_only_eng=pickle.load(open("sub_only_eng.pkl","rb"))

In [0]:
type(sub_only_eng)

list

In [0]:
import pickle

In [0]:
import pandas as pd 

In [0]:
type(X_train)

pandas.core.series.Series

## For converting list to series

In [0]:
sub_only_eng=pd.Series(sub_only_eng) 

In [0]:
lines.shape

(2000, 2)

In [0]:
##LOADING PICKLE FILE OF THE VOCABULARY OF SUBWORD OF ENG AND SUBWORD ENG AND SUBWORD FRENCH

In [0]:
import pickle

In [0]:
sub_only_eng=pickle.load(open("sub_only_eng.pkl","rb"))

In [0]:
all_sub_eng_vocab=pickle.load(open("all_sub_eng_vocab.pkl","rb"))

In [127]:
#LOADING FRENCH FILE AND MAKING THE VOCAB OF FRENCH CHARACTERS
data = open('fre.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))
print(chars)

There are 29880 total characters and 43 unique characters in your data.
['m', 'â', 'o', 'q', 'i', 'ï', 'k', 'f', 's', '«', 'ç', 't', 'c', '\u202f', 'n', 'é', 'ê', 'e', 'x', 'p', '\n', 'z', 'b', 'v', 'a', 'î', 'ô', 'd', 'r', 'à', 'j', 'œ', 'u', '’', 'è', 'g', 'ù', 'û', ' ', 'y', '»', 'l', 'h']


In [128]:
#To get maximum number of the character in a line of the french sentence
file = open('fre.txt', 'r').readlines()
print(len(file))
length = [len(x) for x in file]
print(max(length))

2000
35


In [129]:
#MAKING THE DICTIONARY OF CHARCTER LIST OF FRENCH
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)
print(char_to_ix)

{0: '\n', 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'x', 25: 'y', 26: 'z', 27: '«', 28: '»', 29: 'à', 30: 'â', 31: 'ç', 32: 'è', 33: 'é', 34: 'ê', 35: 'î', 36: 'ï', 37: 'ô', 38: 'ù', 39: 'û', 40: 'œ', 41: '’', 42: '\u202f'}
{'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'x': 24, 'y': 25, 'z': 26, '«': 27, '»': 28, 'à': 29, 'â': 30, 'ç': 31, 'è': 32, 'é': 33, 'ê': 34, 'î': 35, 'ï': 36, 'ô': 37, 'ù': 38, 'û': 39, 'œ': 40, '’': 41, '\u202f': 42}


In [ ]:
#ADDING SUM CHARCTERS TO THE DICTIONARY WHICH WAS NOT PRESENT
char_to_ix['$']=44
char_to_ix['#']=45
char_to_ix['_']=46
ix_to_char[44]='$'
ix_to_char[45]='#'
ix_to_char[46]='_'

In [0]:
# Vocabulary of English sub_words
all_eng_sub_words=set()
for eng in subword_eng:
    print(eng)
    for word in eng:
        print(word)
        if word not in all_eng_sub_words:
            all_eng_sub_words.add(word)


In [0]:
import pickle

In [0]:
import numpy as np

In [130]:
# Max Length of source sequence
lenght_list=[]
for eng in sub_only_eng:
    #print(eng)
    lenght_list.append(len(eng))
    #print(lenght_list)
max_length_src = np.max(lenght_list)

max_length_src

6

In [3]:
#Making the max length same on both encoder and Decoder Side
max_length_src=39
max_length_tar=39

In [133]:
input_words = sorted(list(all_sub_eng_vocab))
target_words = sorted(list(chars))
num_encoder_tokens = len(all_sub_eng_vocab)
num_decoder_tokens = len(chars)
num_encoder_tokens, num_decoder_tokens

(713, 43)

In [0]:
len(input_words)

713

In [134]:
num_decoder_tokens += 4 # For zero padding
num_decoder_tokens

47

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])


In [0]:
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
target_token_index.keys()

dict_keys(['\n', ' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '«', '»', 'à', 'â', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '’', '\u202f'])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())

In [139]:
reverse_input_char_index

{1: 'a',
 2: 'ab',
 3: 'ad',
 4: 'aded',
 5: 'af',
 6: 'aff',
 7: 'ag',
 8: 'ailed',
 9: 'ak',
 10: 'ake',
 11: 'aken',
 12: 'aking',
 13: 'am',
 14: 'ama',
 15: 'amed',
 16: 'ap',
 17: 'ar',
 18: 'ard',
 19: 'ase',
 20: 'astic',
 21: 'asty',
 22: 'at',
 23: 'ate',
 24: 'ated',
 25: 'ating',
 26: 'ats',
 27: 'ax',
 28: 'ay',
 29: 'az',
 30: 'b',
 31: 'ber',
 32: 'ble',
 33: 'body',
 34: 'by',
 35: 'cel',
 36: 'ct',
 37: 'd',
 38: 'ded',
 39: 'ding',
 40: 'e',
 41: 'ed',
 42: 'edy',
 43: 'ell',
 44: 'elled',
 45: 'ely',
 46: 'en',
 47: 'ep',
 48: 'er',
 49: 'ers',
 50: 'es',
 51: 'ess',
 52: 'essed',
 53: 'est',
 54: 'et',
 55: 'ets',
 56: 'ey',
 57: 'fer',
 58: 'fic',
 59: 'ful',
 60: 'get',
 61: 'h',
 62: 'hed',
 63: 'i',
 64: 'ible',
 65: 'ice',
 66: 'iced',
 67: 'ick',
 68: 'id',
 69: 'ide',
 70: 'iding',
 71: 'ied',
 72: 'ies',
 73: 'iful',
 74: 'iled',
 75: 'in',
 76: 'ined',
 77: 'ing',
 78: 'ink',
 79: 'inn',
 80: 'inted',
 81: 'iot',
 82: 'ious',
 83: 'ip',
 84: 'ird',
 85: 'ir

In [140]:
lines = shuffle(lines)
lines.head(10)

,eng,fre
1621,go home now,$_ va à la maison maintenant _#
1434,who saw me,$_ qui ma vu _#
125,hold on,$_ ne quittez pas _#
1016,i like him,$_ je lapprécie _#
143,join us,$_ joignezvous à nous _#
611,im sorry,$_ je suis désolé _#
1227,let him go,$_ laissezle sen aller _#
1700,how are you,$_ comment allonsnous _#
838,ask anyone,$_ demandez à nimporte qui _#
1951,i went too,$_ jy suis également allé _#


In [0]:
# Train - Test Split
X, y = lines.eng,lines.fre
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)


In [0]:
len(y_train)

1800

In [0]:
len(X_test)

200

In [0]:
X_train=pd.Series(X_train)

In [0]:
y_train=pd.Series(y_train) 

In [0]:
y_test=pd.Series(y_test)

In [0]:
type(X_train)

pandas.core.series.Series

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128


In [0]:
def generate_batch(X = X_train, y = y_train, batch_size =128 ):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                subword_eng=bpemb_en.encode(input_text)
                for t, word in enumerate(subword_eng):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                            ch = [ch for ch in word]
                            for ch in ch:
                                decoder_input_data[i, t] = char_to_ix[ch] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                            ch = [ch for ch in word]
                            for ch in ch:
                                decoder_target_data[i, t - 1, char_to_ix[ch]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [0]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [153]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/5
14/14 [==============================] - 8s 539ms/step - loss: 12.9540 - acc: 0.2206 - val_loss: 12.0282 - val_acc: 0.2444
Epoch 2/5
14/14 [==============================] - 2s 128ms/step - loss: 11.3923 - acc: 0.2319 - val_loss: 10.6712 - val_acc: 0.2125
Epoch 3/5
14/14 [==============================] - 2s 129ms/step - loss: 10.7579 - acc: 0.2182 - val_loss: 10.5912 - val_acc: 0.2136
Epoch 4/5
14/14 [==============================] - 2s 129ms/step - loss: 10.4279 - acc: 0.2059 - val_loss: 10.1696 - val_acc: 0.1905
Epoch 5/5
14/14 [==============================] - 2s 130ms/step - loss: 10.2920 - acc: 0.2061 - val_loss: 10.2134 - val_acc: 0.2012


### Always remember to save the weights

In [0]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application